In [21]:
import os,sys
import logging, time
from pprint import pprint, pformat
import importlib
from datetime import timedelta, datetime
import pandas as pd
import numpy as np
import qgrid
import plotly.offline as offline
import plotly.tools as tools, plotly.graph_objs as go
from docs.config import *
from docs.data import *
import app
from app import freqtostr, strtofreq, pertostr
from app.common.timer import Timer
from app.common.utils import utc_datetime as now, utc_dtdate
from app.bnc import scanner, candles, signals, trade, strategy

# Conf
log = logging.getLogger("testing")
pd.set_option("display.max_columns", 25)
pd.set_option("display.width", 2000)
app.set_db(["localhost", "45.79.176.125"][0])
db = app.get_db()

# Generate plot data
freq=300
freq_str='5m'
pair='DGDBTC'
trade.freq = freq
trade.freq_str = freq_str
trade.init()
dfc = app.bnc.dfc
btc = dfc.loc[pair,freq].tail(288)
macd = signals.macd(btc, 12, 26)
qdf = qgrid.show_grid(macd,
    show_toolbar=True, grid_options={'forceFitColumns': True, 'defaultColumnWidth': 100})

# Offline plot
fig = tools.make_subplots(
    rows=4, cols=1,
    specs=[ [{}], [{}], [{}], [{}] ], 
    shared_xaxes=True, shared_yaxes=True, vertical_spacing=0.001)
fig.append_trace(go.Scatter(x=macd.index,y=macd['close']), 1, 1)

fig.append_trace(go.Scatter(x=macd.index, y=macd['macd'], name="MACD"), 2, 1)
fig.append_trace(go.Bar(x=macd.index, y=macd['macd_diff'], name="MACD_diff"), 2, 1)
fig.append_trace(go.Scatter(x=macd.index, y=macd['macd_sign'], name="MACD_sign"), 2, 1)


fig.append_trace(go.Bar(x=macd.index, y=macd['volume'], name="Volume"), 3, 1)

fig.append_trace(go.Scatter(x=macd.index, y=macd['buy_ratio'], name="Buy Ratio"), 4, 1)

fig['layout'].update(title='{} MACD'.format(pair))

offline.plot(fig, filename="myplot.html")

KeyError: 'the label [DGDBTC] is not in the [index]'